<font size=4>As I get closer to the grandmaster tier, I want to take this opportunity to thank this amazing community which has taught me so much about data science and life in general. I also want to thank everyone who made this journey so beautiful and memorable!</font>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

import os
import gc
import re
import folium
import textstat
from scipy import stats
from colorama import Fore, Back, Style, init

import math
import numpy as np
import scipy as sp
import pandas as pd

import random
import networkx as nx
from pandas import Timestamp

from PIL import Image
from IPython.display import SVG
# from keras.utils import model_to_dot

import requests
from IPython.display import HTML

import seaborn as sns
from tqdm import tqdm
import matplotlib.cm as cm
import matplotlib.pyplot as plt

tqdm.pandas()

import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from plotly.subplots import make_subplots

import transformers

from sklearn import metrics
from sklearn.utils import shuffle
from gensim.models import Word2Vec
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.feature_extraction.text import TfidfVectorizer,\
                                            CountVectorizer,\
                                            HashingVectorizer

from nltk.stem.wordnet import WordNetLemmatizer 
from nltk.tokenize import word_tokenize
from nltk.tokenize import TweetTokenizer  

import nltk
from textblob import TextBlob

from nltk.corpus import wordnet
from nltk.corpus import stopwords
from googletrans import Translator
from nltk import WordNetLemmatizer
from polyglot.detect import Detector
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud, STOPWORDS
from nltk.sentiment.vader import SentimentIntensityAnalyzer

stopword=set(STOPWORDS)

lem = WordNetLemmatizer()
tokenizer=TweetTokenizer()

np.random.seed(0)

In [ ]:
import icu
from polyglot.text import Text, Word

In [ ]:
DATA_PATH = "../input/jigsaw-multilingual-toxic-comment-classification/"
os.listdir(DATA_PATH)

In [ ]:
TEST_PATH = DATA_PATH + "test.csv"
VAL_PATH = DATA_PATH + "validation.csv"
TRAIN_PATH = DATA_PATH + "jigsaw-toxic-comment-train.csv"

val_data = pd.read_csv(VAL_PATH)
test_data = pd.read_csv(TEST_PATH)
train_data = pd.read_csv(TRAIN_PATH)

In [ ]:
def get_language(text):
    return Detector("".join(x for x in text if x.isprintable()), quiet=True).languages[0].name

train_data["lang"] = train_data["comment_text"].progress_apply(get_language)

In [ ]:
lang_list = sorted(list(set(train_data["lang"])))
counts = [list(train_data["lang"]).count(cont) for cont in lang_list]
df = pd.DataFrame(np.transpose([lang_list, counts]))
df.columns = ["Language", "Count"]
df["Count"] = df["Count"].apply(int)

df_en = pd.DataFrame(np.transpose([["English", "Non-English"], [max(counts), sum(counts) - max(counts)]]))
df_en.columns = ["Language", "Count"]

fig = px.bar(df_en, x="Language", y="Count", title="Language of comments", color="Language", text="Count")
fig.update_layout(template="plotly_white")
fig.data[0].marker.line.color = 'rgb(0, 0, 0)'
fig.data[0].marker.line.width = 0.5
fig.data[1].marker.line.color = 'rgb(0, 0, 0)'
fig.data[1].marker.line.width = 0.5
fig.data[0].textfont.color = "black"
fig.data[0].textposition = "outside"
fig.data[1].textfont.color = "black"
fig.data[1].textposition = "outside"
fig

In [ ]:
fig = px.bar(df.query("Language != 'English' and Language != 'un'").query("Count >= 50"),
             y="Language", x="Count", title="Language of non-English comments", template="plotly_white", color="Language", text="Count", orientation="h")
fig.update_traces(marker=dict(line=dict(width=0.75,
                                        color='black')),  textposition="outside")
fig.update_layout(showlegend=False)
fig

In [ ]:
fig = go.Figure([go.Pie(labels=df.query("Language != 'English' and Language != 'un'").query("Count >= 50")["Language"],
           values=df.query("Language != 'English' and Language != 'un'").query("Count >= 50")["Count"])])
fig.update_layout(title_text="Pie chart of non-English languages", template="plotly_white")
fig.data[0].marker.colors = [px.colors.qualitative.Plotly[2:]]
fig.data[0].textfont.color = "black"
fig.data[0].textposition = "outside"
fig.show()

In [ ]:
def new_len(x):
    if type(x) is str:
        return len(x.split())
    else:
        return 0

train_data["comment_words"] = train_data["comment_text"].apply(new_len)
nums = train_data.query("comment_words != 0 and comment_words < 200").sample(frac=0.1)["comment_words"]
fig = ff.create_distplot(hist_data=[nums],
                         group_labels=["All comments"],
                         colors=["coral"])

fig.update_layout(yaxis_title="Frequncy",xaxis_title="Comment words", template="simple_white", showlegend=False)

fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_comment_words.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
df = pd.DataFrame(np.transpose([lang_list, train_data.groupby("lang").mean()["comment_words"]]))
df.columns = ["Language", "Average_comment_words"]
df["Average_comment_words"] = df["Average_comment_words"].apply(float)
df = df.query("Average_comment_words < 500")
fig = go.Figure(go.Bar(x=df["Language"], y=df["Average_comment_words"]))

# fig.update_layout(xaxis_title="Language", yaxis_title="Average comment words", title_text="Average comment words vs. language", template="plotly_white")
fig.update_layout(xaxis_title="Language", yaxis_title="Average comment words", template="plotly_white")

fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_comment_words_language.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))

fig.show()

In [ ]:
def polarity(x):
    if type(x) == str:
        return SIA.polarity_scores(x)
    else:
        return 1000
    
SIA = SentimentIntensityAnalyzer()
train_data["polarity"] = train_data["comment_text"].progress_apply(polarity)

In [ ]:
fig = go.Figure(go.Histogram(x=[pols["neg"] for pols in train_data["polarity"] if pols["neg"] != 0], marker=dict(
            color='seagreen')
    ))

# fig.update_layout(xaxis_title="Negativity sentiment", title_text="Negativity sentiment", template="simple_white")

fig.update_layout(yaxis_title="Count",xaxis_title="Negativity sentiment", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_neg_sentence.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
train_data["negativity"] = train_data["polarity"].apply(lambda x: x["neg"])


In [ ]:
nums_1 = train_data.sample(frac=0.1).query("toxic == 1")["negativity"]
nums_2 = train_data.sample(frac=0.1).query("toxic == 0")["negativity"]

fig = ff.create_distplot(hist_data=[nums_1, nums_2],
                         group_labels=["Toxic", "Non-toxic"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

# fig.update_layout(title_text="Negativity vs. Toxicity", xaxis_title="Negativity", template="simple_white")
fig.update_layout(yaxis_title="Count",xaxis_title="Negativity", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_neg_toxic.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
train_data["positivity"] = train_data["polarity"].apply(lambda x: x["pos"])

In [ ]:
nums_1 = train_data.sample(frac=0.1).query("toxic == 1")["positivity"]
nums_2 = train_data.sample(frac=0.1).query("toxic == 0")["positivity"]

fig = ff.create_distplot(hist_data=[nums_1, nums_2],
                         group_labels=["Toxic", "Non-toxic"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

fig.update_layout(yaxis_title="Count",xaxis_title="Positivity", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_pos_toxic.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
fig = go.Figure(go.Histogram(x=[pols["neu"] for pols in train_data["polarity"] if pols["neu"] != 1], marker=dict(
            color='dodgerblue')
    ))

fig.update_layout(yaxis_title="Count",xaxis_title="Neutrality sentiment", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_neu_sentence.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
train_data["neutrality"] = train_data["polarity"].apply(lambda x: x["neu"])


In [ ]:
nums_1 = train_data.sample(frac=0.1).query("toxic == 1")["neutrality"]
nums_2 = train_data.sample(frac=0.1).query("toxic == 0")["neutrality"]

fig = ff.create_distplot(hist_data=[nums_1, nums_2],
                         group_labels=["Toxic", "Non-toxic"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

fig.update_layout(yaxis_title="Count",xaxis_title="Neutrality", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_neu_toxic.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
fig = go.Figure(go.Histogram(x=[pols["compound"] for pols in train_data["polarity"] if pols["compound"] != 0], marker=dict(
            color='orchid')
    ))

fig.update_layout(yaxis_title="Count",xaxis_title="Compound sentiment", template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_all_sentence.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()

In [ ]:
train_data["compound"] = train_data["polarity"].apply(lambda x: x["compound"])


In [ ]:
nums_1 = train_data.sample(frac=0.1).query("toxic == 1")["compound"]
nums_2 = train_data.sample(frac=0.1).query("toxic == 0")["compound"]

fig = ff.create_distplot(hist_data=[nums_1, nums_2],
                         group_labels=["Toxic", "Non-toxic"],
                         colors=["darkorange", "dodgerblue"], show_hist=False)

fig.update_layout(xaxis_title="Compound",yaxis_title="Count",template="simple_white")
fig.update_layout(
    autosize=True,
    width=1000,
    height=400,)
picpath="pic/3_all_toxic.png"
fig.write_image(picpath)
print('![]({})'.format(picpath))
fig.show()